<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Train Practice

## *Data Science Unit 4 Sprint 2 Assignment 4*

Continue to use TensorFlow Keras & a sample of the [Quickdraw dataset](https://github.com/googlecreativelab/quickdraw-dataset) to build a sketch classification model. The dataset has been sampled to only 10 classes and 10000 observations per class. Apply regularization techniques to your model. 

*Don't forgot to switch to GPU on Colab!*

## Regularization

Using your best performing model from the previous module, apply each of the following regularization strategies: 
* Early Stopping
* Dropout
* Weight Decay
* Weight Constraint


In [ ]:
# Your Code Starts Here

## Deploy

Save your model's weights using the Checkpoint function. Try reloading the model and making inference on your validation dataset.

In [ ]:
# Your Code Starts Here

### Stretch Goals
- Mount your Google Drive to Colab to persist your model checkpoint files. 
- Research L2 normalization (weight decay)
- Write a custom callback function to stop training after you reach .88 validation accuracy. 
- Select a new dataset and apply a neural network to it.
- Research TensorFlow Serving
- Play [QuickDraw](https://quickdraw.withgoogle.com/data)
- Create a static webpage using TensorFlow.js to serve a model. Check out [Teachable Machine Learning](https://teachablemachine.withgoogle.com/) for ideas. 